In [7]:
import re
import urllib.parse

from pymongo import MongoClient
from sqlalchemy import ForeignKeyConstraint
from sqlalchemy import create_engine, Column, BigInteger, Float, String, Enum, ForeignKey, MetaData
from sqlalchemy.dialects import postgresql
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.sql.ddl import CreateTable
from sqlalchemy.sql.sqltypes import DateTime, Date

In [8]:
db_config = {
    'postgres': {
        'host': 'localhost',
        'database': 'temp',
        'username': 'postgres',
        'password': 'postgres',
    },
    'mongo': {
        'host': 'localhost',
        'port': '27017',
        'database': 'tenshi',
        'username': 'root',
        'password': 'root',
        'auth_source': 'admin'
    }
}

In [ ]:
def get_mongo_connection(mongo_config):
    """
    Create a MongoDB connection string from the configuration dictionary.
    """
    username = urllib.parse.quote_plus(mongo_config['username'])
    password = urllib.parse.quote_plus(mongo_config['password'])
    connection_string = (f"mongodb://{username}:{password}@"
                         f"{mongo_config['host']}:{mongo_config['port']}/"
                         f"{mongo_config['database']}?authSource={mongo_config['auth_source']}")
    return MongoClient(connection_string)


def get_postgres_engine(postgres_config):
    username = urllib.parse.quote_plus(postgres_config['username'])
    password = urllib.parse.quote_plus(postgres_config['password'])
    connection_string = (f"postgresql+psycopg2://{username}:{password}@"
                         f"{postgres_config['host']}/"
                         f"{postgres_config['database']}")
    return create_engine(connection_string, echo=False)


# Function to convert camel case to snake case
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()
    # Then, remove any non-alphanumeric characters (except underscores)
    # by keeping only characters that are alphanumeric or underscores
    name = re.sub(r'[^a-zA-Z0-9_]', '', name)
    return name


In [ ]:
mongo_client = get_mongo_connection(db_config['mongo'])
source = mongo_client[db_config['mongo']['database']]
destination_engine = get_postgres_engine(db_config['postgres'])

In [6]:


from sqlalchemy.schema import ForeignKeyConstraint
from sqlalchemy.sql.ddl import AddConstraint


def create_rdbms_scheme(documents):
    # A dictionary to hold all your table classes, since they're dynamically created
    table_classes = {}

    # A list to hold foreign key relations
    foreign_keys = []

    # A list to hold relations for alter table statements
    relations_for_alter = []

    # Base class for declarative tables
    Base = declarative_base()

    # Create a PostgreSQL table for each MongoDB document
    for doc in documents:
        table_name = camel_to_snake(doc['externalId'])
        class_name = camel_to_snake(table_name)

        primary_key_column_name = f"{table_name}_id"  # format the primary key as 'table_name_id'
        # Create a new type dynamically
        table_class = type(class_name, (Base,), {
            '__tablename__': table_name,
            primary_key_column_name: Column(String, primary_key=True),
            '__table_args__': {'extend_existing': True}  # Allow redefinition if rerunning
        })

        # Add properties as columns
        for prop in doc['properties']:
            col_name = camel_to_snake(prop['externalId'])
            col_type = String  # Default type is String

            if prop['inputType'] == 'NUMBER':
                col_type = Float
            elif prop['inputType'] == 'DATE_TIME':
                col_type = DateTime
            elif prop['inputType'] == 'DATE':
                col_type = Date
            elif 'options' in prop:  # Enum type
                # assuming the dictionaries have a 'name' field you want to use for the Enum
                option_names = [option['displayName'] for option in prop['options']]
                col_type = Enum(*option_names, name=f"{table_name}_{col_name}_enum")

            comment = f"{prop['_id']}"
            setattr(table_class, col_name, Column(col_type, comment=comment))

        # Save the table class to the dictionary
        table_classes[table_name] = table_class

    # Now handle relations
    for doc in documents:
        table_name = camel_to_snake(doc['externalId'])
        table_class = table_classes[table_name]

        for rel in doc.get('relations', []):
            # Find the related table
            related_table_name = camel_to_snake(rel['externalId'])
            related_table_class = table_classes.get(related_table_name)


            if related_table_class:
                # Prepare the information needed for alter table statements
                source_table_name = camel_to_snake(doc['externalId'])
                source_column_name = f"{related_table_name}_id"
                target_table_name = related_table_name
                target_column_name = f"{related_table_name}_id"
            related_table_class = table_classes.get(related_table_name)

            if related_table_class:
                # Prepare the informat
                relations_for_alter.append({
                    'source_table': source_table_name,
                    'source_column': source_column_name,
                    'target_table': target_table_name,
                    'target_column': target_column_name
                })

            # if related_table_class:
            #     # Create a new column for the foreign key
            #     related_primary_key = f"{related_table_name}_id"
            #     foreign_key_column = f"{related_table_name}_id"
            #     foreign_key = ForeignKey(f"{related_table_name}.{related_primary_key}")
            #     setattr(table_class, foreign_key_column, Column(String, foreign_key))
            # 
            #     # Add this relation to the foreign_keys list for creating the constraints later
            #     foreign_keys.append({
            #         'source_table': table_name,
            #         'source_column': foreign_key_column,
            #         'target_table': related_table_name,
            #         'target_column': related_primary_key
            #     })

    # Create all tables in the database
    Base.metadata.create_all(destination_engine)

    # Print the SQL statements for alter table add foreign key constraints
    for rel in relations_for_alter:
        alter_stmt = (
            f"ALTER TABLE {rel['source_table']} "
            f"ADD CONSTRAINT fk_{rel['source_table']}_{rel['source_column']} "
            f"FOREIGN KEY ({rel['source_column']}) "
            f"REFERENCES {rel['target_table']}({rel['target_column']});"
        )
        print(alter_stmt)
    # Add foreign keys 
    # metadata = MetaData()
    # metadata.reflect(bind=destination_engine)
    # 
    # # Add constraints to database
    # with destination_engine.connect() as conn:
    #     for fk in foreign_keys:
    #         src_table = metadata.tables[fk['source_table']]
    #         tgt_table = metadata.tables[fk['target_table']]
    # 
    #         constraint = ForeignKeyConstraint(
    #             [src_table.columns[fk['source_column']]],
    #             [tgt_table.columns[fk['target_column']]],
    #             name=f"fk_{fk['source_table']}_{fk['source_column']}"
    #         )
    #         src_table.append_constraint(constraint)
    #         conn.execute(AddConstraint(constraint))

collection = source['objectTypes']
object_types = collection.find({'usageStatus': 1}).sort('externalId', 1)
create_rdbms_scheme(list(object_types))

/tmp/ipykernel_22774/1068334155.py:16: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


KeyError: 'areas_id'

In [ ]:
collection = source['objectTypes']
object_types = collection.find({'usageStatus': 1}).sort('externalId', 1)
create_rdbms_scheme(list(object_types))